# init spark session

In [6]:
from IPython.core.display import HTML
display(HTML("""<style>pre { white-space: pre !important; }.container { width:100% !important; }</style>"""))

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("staging-to-rawvault-streaming")
    .config("spark.driver.memory", "1g")
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse/default")
    .config("spark.jars", """
        hdfs://namenode:9000/user/hive/spark_jars/iceberg-hive-runtime-1.4.3.jar,
        hdfs://namenode:9000/user/hive/spark_jars/iceberg-spark-runtime-3.4_2.12-1.4.3.jar
    """)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.executor.cores", 3)
    .config("spark.executor.memory", "6g")
    .config("spark.executor.instances", 2)
    .enableHiveSupport()
    .getOrCreate()
)

24/11/17 09:31:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

# check staging

In [14]:
path = 'hdfs://namenode:9000/user/hive/warehouse/staging/topics/product1.public.customer/'
df = (
    spark.read.format('parquet')
    .options(header=True,inferSchema=True)
    .load(path)
)
df.show(10, False)

+-----------+----------+---------+----------+-----------------+------------+------------------+-----------------+----------------+----+--------+-------------+---------+
|customer_id|first_name|last_name|birth_date|address          |phone_number|email             |job_title        |updated_datetime|__op|__lsn   |__src_ts_ms  |partition|
+-----------+----------+---------+----------+-----------------+------------+------------------+-----------------+----------------+----+--------+-------------+---------+
|1          |Vu        |Tran     |18583     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731864517000000|u   |26934384|1731839317757|0        |
|1          |Vu        |Tran     |11278     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731865025000000|u   |26935792|1731839825873|0        |
|1          |Vu        |Tran     |11643     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731865031000000|u   |26937064|17318398314

In [16]:
# spark.sql(f"""
# create external table staging.product1_customer
# ({', '.join([col + ' ' + dtype for col, dtype in df.dtypes])})
# stored as parquet
# location "hdfs://namenode:9000/user/hive/warehouse/staging/topics/product1.public.customer"
# """)

In [18]:
spark.sql(f"""
select *
from staging.product1_customer
order by __lsn desc
""").show(10, False)

+-----------+----------+---------+----------+-----------------+------------+------------------+-----------------+----------------+----+--------+-------------+---------+
|customer_id|first_name|last_name|birth_date|address          |phone_number|email             |job_title        |updated_datetime|__op|__lsn   |__src_ts_ms  |partition|
+-----------+----------+---------+----------+-----------------+------------+------------------+-----------------+----------------+----+--------+-------------+---------+
|1          |Vu        |Tran     |11643     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731865031000000|u   |26937064|1731839831416|0        |
|1          |Vu        |Tran     |11278     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731865025000000|u   |26935792|1731839825873|0        |
|1          |Vu        |Tran     |18583     |123 ABC, MNO, XYZ|0865937123  |nitsvutt@gmail.com|Data Engineer    |1731864517000000|u   |26934384|17318393177